# AptaNet Tutorial
Step-by-step guide to using AptaNet for binary aptamer–protein binding prediction.

## Overview
- **pairs_to_features**: converts `(aptamer_seq, protein_seq)` pairs into feature vectors using k-mer + PSeAAC.
- **FeatureSelector**: a Random Forest-based transformer that selects important features.
- **SkorchAptaNet**: a PyTorch MLP wrapped in Skorch for binary classification with a configurable threshold.
- **load_pfoa_structure**: helper to load PFOA molecule structure from PDB file.

## Imports
Import the core functions and classes—you don’t need to redefine them.

In [ ]:
import torch
import torch.optim as optim
from sklearn.pipeline import Pipeline

from pyaptamer.aptanet import FeatureSelector, SkorchAptaNet
from pyaptamer.utils._aptanet_utils import load_pfoa_structure, pairs_to_features

## Pipeline
Build a scikit-learn pipeline combining feature extraction, selection, and classification. If you already have a `pipe` defined in your project, you can import and use that instead.

In [ ]:
# Define the classifier
net = SkorchAptaNet(
    module__hidden_dim=128,
    module__n_hidden=7,
    module__dropout=0.3,
    max_epochs=200,
    lr=1.4e-4,
    batch_size=310,
    optimizer=optim.RMSprop,
    device="cuda" if torch.cuda.is_available() else "cpu",
    threshold=0.5,
    verbose=1,
)

# Option 1: build a new pipeline here
pipe = Pipeline(
    [
        ("features", pairs_to_features),
        ("selector", FeatureSelector()),
        ("clf", net),
    ]
)

# Option 2: import the pre-defined pipeline

## Loading PFOA Structure
Use the provided loader to get the PFOA structure for visualization or custom feature work:

In [ ]:
structure = load_pfoa_structure()
structure

## Usage Example
Once you have your dataset:
```python
# X: list of (aptamer_seq, protein_seq)
# y: binary labels (0/1)
pipe.fit(X, y)
preds = pipe.predict(X)
probs = pipe.predict_proba(X)[:, 1]
print(classification_report(y, preds))
print('ROC AUC:', roc_auc_score(y, probs))
```